## TensorFlow:
TensorFlow is the most popular and adopted free and open-source deep learning library. It was first developed and maintained by Google. It can be used for both research and production.

## **TensorFlow benefits:**
- Highly efficient
- Cross-platform (works on IOS, Android, Unix, Windows, in the cloud, in the browser etc etc)
- Calculates gradients automatically (this is truly useful for Neural Networks, where the analytical solution of gradients would be VERY tedious to derive).
* Deep integration with the Keras library (Functional approach, as well as high-level wrapper)

# General notebook setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# Hide warnings
import warnings
warnings.filterwarnings('ignore')

# Install TensorFlow 2.0

TensorFlow 2.x is a major change from TensorFlow 1.x (not backwards compatible, however you can use a tool to convert your TensorFlow 1.x code to 2.x).

The new version is designed to be more pythonic. It's  easier to debug models, extract values during training (because of the need of sessions and graphs in TensorFlow 1.x). 

TensorFlow 2.x supports eager execution by default, so you don't need a session and to evaluate operations / tensors in order to extract values.

In [ ]:
#!pip install tensorflow
# or for GPU version:
# !pip install tensorflow-gpu

# Import TensorFlow

In [2]:
# Canonical way of importing TensorFlow
import tensorflow as tf

# If this doesn't work TensorFlow is not installed correctly

# TensorFlow 2.0
At the time of the update of this notebook we are still in the early days of TensorFlow, and currently (Oct 22) the version 2.0.0 has just been released.

In [3]:
# Check tf version, oftentimes tensorflow is not backwards compatible
tf.__version__

# should be tensorflow 2

'2.0.0'

# Intro to TensorFlow
### Core components:

#### 1. Tensor
A Tensor in TensorFlow is an N-dimensional array (just like Numpys array object). Tensors are multilinear maps from vector spaces to real numbers. Scalars, vectors and matrices are all tensors. The Tensor represents units of data in TensorFlow.

Numpy arrays or Pandas DataFrames sent to Tensorflow functions are automatically converted into TensorFlow tensors.

#### 2. Operations / Ops
TensorFlow operations or ops are units / edges / nodes of computation (e.g. matrix multiplication, addition, etc.)

#### 3. Computation Graph
The computational graph is is an optimized, compiled representation of the dataflow and the order of computations that are sent to an execution environment (for example during model training).

TensforFlow 2.x supports eager execution, but when we build a model and then train it TensorFlow can compile the model and optimize the executions as a computational graph object. This is done by decorating a function with `@tf.function`.

This computational graph is then  sent to another instance / runtime environment (e.g. on a CPU or GPU) for execution. The results are sent back to us. This makes TensorFlow computations highly distributable and it also allows us to automatically evaluate all gradients in the computation nodes.

![](imgs/tf_graph.png)

TensorFlow 2.x supports eager execution by default.

In [4]:
tf.executing_eagerly() 

True

In [14]:
a = tf.Variable(20)

In [15]:
type(a)

tensorflow.python.ops.resource_variable_ops.ResourceVariable

# TensorBoard setup
Tip2: Setup TensorBoard if you want to monitor and analyze computational graphs etc.

In [16]:
from datetime import datetime
import os
import pathlib

t = datetime.utcnow().strftime("%Y%m%d%H%M%S") 
log_dir = "tf_logs"
logd = "{}/r{}/".format(log_dir, t)

# Make directory if it doesn't exist

from pathlib import Path
home = str(Path.home())

logdir = os.path.join(os.sep,home,logd)

if not os.path.exists(logdir):
    os.makedirs(logdir)

In [17]:
logdir

'/Users/anuj/tf_logs/r20200718142227/'

# 1. TensorFlow tensors

## 1.1 tf.constant

Constants are initialized directly and eager execution let's us see the values without creating a session and running the tensor.

In [18]:
a = tf.constant(2)
b = tf.constant(5)

In [19]:
a # note the numpy value

<tf.Tensor: id=27, shape=(), dtype=int32, numpy=2>

The .numpy() method will return the result as a numpy array.

In [20]:
# Eager evaluation of tensors
a.numpy()

2

### We can also perform operations on tensors

In [21]:
a*b

<tf.Tensor: id=29, shape=(), dtype=int32, numpy=10>

#### or the same with universal functions

In [22]:
tf.multiply(a,b).numpy()

10

In [23]:
a_matrix = tf.constant([[1,2], [3,4]])
b_matrix = tf.constant([[5,6], [7,8]])
b_matrix

<tf.Tensor: id=32, shape=(2, 2), dtype=int32, numpy=
array([[5, 6],
       [7, 8]], dtype=int32)>

In [24]:
tf.matmul(a_matrix, b_matrix)

<tf.Tensor: id=33, shape=(2, 2), dtype=int32, numpy=
array([[19, 22],
       [43, 50]], dtype=int32)>

##### Note, we cannot reassign values of constants (like we can with Variables).

In [26]:
print(a_matrix, b_matrix, sep="\n")

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[5 6]
 [7 8]], shape=(2, 2), dtype=int32)


## 1.2 tf.Variable

Variables are mutable and can be updated and reassigned new values. Variables are usually weights and biases of a model that are optimized during training, they also indicate the degrees of freedom of the model (what model parameters that can change, thus making the model flexible).

In [29]:
var = tf.Variable(3.)
var

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=3.0>

In [30]:
# Reassign the value of a Variable
var.assign(4)
var

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=4.0>

In [ ]:
var.numpy()

In [27]:
# we can also create multi dim Variables.
d = tf.Variable(np.random.randn(3, 3)) #reshape
# automatically assings data type
d #

<tf.Variable 'Variable:0' shape=(3, 3) dtype=float64, numpy=
array([[-0.65163397,  1.58152426,  0.53522323],
       [-1.70599492,  0.98212383, -0.10321037],
       [-1.38010494, -0.09607557,  0.24267402]])>

In [31]:
# inplace increase / decrease Variable values

var.assign(10)
print('original value:', var.numpy())
print('add 1:', var.assign_add(1.).numpy())
print('subtract 5:', var.assign_sub(5.).numpy())

original value: 10.0
add 1: 11.0
subtract 5: 6.0


### Variables also have a lot of attributes associated with them:

In [32]:
v = tf.Variable([[3.,3.2], [1.2,2.2]], dtype=tf.float32, name='my_variable')

print('name  : ', v.name)
print('type  : ', v.dtype)
print('shape : ', v.shape)
print('device: ', v.device)

name  :  my_variable:0
type  :  <dtype: 'float32'>
shape :  (2, 2)
device:  /job:localhost/replica:0/task:0/device:CPU:0


<div class='alert alert-info'><b>Note</b>: Tensorflow is really similar to NumPy, and you can think of the tensors as an ndimensional array.</div>


![tf_to_np](imgs/tf_to_np.png)
Source: CS227d, NLP, Stanford

# 2. Operations / Ops
Operations can be carried out directly or assigned to variables.

In [33]:
op1 = tf.add(a,b)
op1

<tf.Tensor: id=65, shape=(), dtype=int32, numpy=7>

In [34]:
a+b # same as tf.add

<tf.Tensor: id=66, shape=(), dtype=int32, numpy=7>

In [35]:
v = a+b
u = v+2
w = v*u
z = w*3
z

<tf.Tensor: id=72, shape=(), dtype=int32, numpy=189>

## Look at the computational graph with @tf.function

`@tf.function` is a very useful module that can be used to convert simple python functions into a highly optimized computational graph that can be run on any runtime environment. When we build a model and then train it TensorFlow we can compile the model and optimize the executions.

In [49]:
@tf.function
def func2(a,b):
        z = tf.multiply(a,b, name='z')
        y1 = tf.constant(3, name='3')
        y2 = tf.constant(4)
        w1 = tf.add(z, y1, name='w1')
        w2 = tf.add(z, y2, name='w2')
        
        return(w1+w2)

In [41]:
func(10, 20)

<tf.Tensor: id=101, shape=(), dtype=int32, numpy=407>

In [42]:
@tf.function
def func(a,b):
    with tf.name_scope('first'):
        z = tf.multiply(a,b, name='z')
    with tf.name_scope('second'):
        y1 = tf.constant(3, name='3')
        y2 = tf.constant(4)
        w1 = tf.add(z, y1, name='w1')
        w2 = tf.add(z, y2, name='w2')
        
    return(w1+w2)

In [51]:
# Setup a writer to save graph information and TensorFlow logs
# To be displayed with Tensorboard

writer = tf.summary.create_file_writer(logdir)
tf.summary.trace_on()

In [52]:
a = tf.constant(3)
b = tf.constant(4)
func(a,b)
with writer.as_default():
    tf.summary.trace_export(
        name="func2",
        step=0,
        profiler_outdir=logdir)

In [53]:
logdir

'/Users/anuj/tf_logs/r20200718142227/'

In [54]:
# run tensorboard in the shell
!tensorboard --logdir $logdir

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.0.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


### tf.function and Conditional statements
It is difficult to use conditions in graphs but we could implement that easily using `@tf.function` decorator 

In [47]:
@tf.function 
def g(x):
    y = tf.reduce_sum(x)
    if y > 0:
        return y
    return tf.abs(y)

In [ ]:
print(tf.autograph.to_code(g.python_function))

# Calculate gradients

Gradient evaluation is very importnat machine learning because it is based on function optimization. You can use `tf.GradientTape()` method to record the gradient of an arbitrary function

In [56]:
w = tf.Variable(3.0)

# Gradient scope for the function w^2
with tf.GradientTape() as tape:
    expression = tf.exp(w) + 2 * w +  w * w

grad = tape.gradient(expression, w)
print(f'The gradient of exp at {w.numpy()} is {grad.numpy()}')

The gradient of exp at 3.0 is 28.08553695678711


### Gradient of the Sigmoid function
In this example we evaluate the gradient of the sigmoid function 

$$\sigma(x) = \frac{1}{1+e^{-x}}$$

Note that 

$$\sigma'(x) = \frac{e^{-x}}{(1+e^{-x})^2} = \sigma(x)(1-\sigma(x)) $$

For instance 

$$\sigma'(0) = \sigma(0)(1-\sigma(0)) = \frac{1}{2}\left(1-\frac{1}{2} \right) = \frac{1}{4}$$

In [57]:
def sigmoid(x):
    return 1/(1 + tf.exp(-x))

In [58]:
#define a varaible
x = tf.Variable(0.)

#record the gradient
with tf.GradientTape() as tape:
    sig = sigmoid(x)
    
res = tape.gradient(sig, x).numpy()
print('The gradient of the sigmoid function at 0.0 is ', res)

The gradient of the sigmoid function at 0.0 is  0.25
